#Project 3: Dynamic programming
JeongHo Choi, 18th June 2022 (updated)

-find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that. If your function is much slower, there is a problem somewhere.<br>
Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.<br>
Hint 3: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called overlap(a, b, min_length=k) on every pair of reads.

###Question 3)
Picture the overlap graph corresponding to the overlaps just calculated. How many edges are in the graph? In other words, **how many distinct pairs of reads overlap**?

###Question 4) 
Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge? (**In other words, how many reads have a suffix involved in an overlap?**)

In [1]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [2]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [3]:
def overlap_all_pairs(reads, k):
    kmers_dict = {} #create a dictionary for kmers; k-mer as a key, corresponding reads as value
    read_pairs_dict = {} 
    #create a dictionary for read pairs having a suffix involved in an overlap;read pairs as a key, overlap length as value

    for read in reads:
        for i in range(len(read)-k+1):
            kmer=read[i:i+k]
            if kmer not in kmers_dict:
                kmers_dict[kmer]= [read]
            else:
                kmers_dict[kmer].append(read)
    
    pairs_count=0
    for read in reads:
        read_suffix = read[-k:] 
        for possible_read in kmers_dict[read_suffix]:
            if possible_read != read:
                olen = overlap(read, possible_read, k)
                if olen > 0:
                    pairs_count += 1
                    read_pairs_dict[(read, possible_read)] = olen #olen refers to overlap length
                    
    return read_pairs_dict, pairs_count

In [4]:
reads_filename = 'ERR266411_1.for_asm.fastq'
reads, _ = readFastq(reads_filename)
read_pairs, pairs_number = overlap_all_pairs(reads, 30)

In [5]:
# Q3
print("Number of pairs of reads overlap:", pairs_number)

Number of pairs of reads overlap: 904746


In [9]:
read_pairs_keys = []
for key, value in read_pairs:
    read_pairs_keys.append(key)
    
# Q4
print("Number of reads having a suffix involved in an overlap:", len(set(read_pairs_keys)))
#coverting to set in order to exclude repeated read_pairs

Number of reads having a suffix involved in an overlap: 7161
